# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: int
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.1193984107273764),
 (1, 1.1193984107273764),
 (2, 1.1193984107273764),
 (3, 1.1193984107273764),
 (4, 1.1193984107273764)]

## Inverted index 

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('a', 2), ('banana', 2)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.0037556706367033454),
   (None, 0.019078804267606442),
   (None, 0.030079553154343963),
   (None, 0.04771773103294796),
   (None, 0.06489912842832446),
   (None, 0.07807386268652028),
   (None, 0.08289436562960917),
   (None, 0.12035779707561023),
   (None, 0.16159983838231695),
   (None, 0.17717555902084237),
   (None, 0.18831766759399005),
   (None, 0.22430316536968264),
   (None, 0.24101986061224923),
   (None, 0.31134683863435964),
   (None, 0.45577775575102963),
   (None, 0.47703157042549416)]),
 (1,
  [(None, 0.5256985372804456),
   (None, 0.5651713536257229),
   (None, 0.574661839141784),
   (None, 0.59534719015213),
   (None, 0.6444576743347876),
   (None, 0.6483232771627858),
   (None, 0.7181022955874682),
   (None, 0.7325432768577975),
   (None, 0.7791224317126139),
   (None, 0.7847754034852154),
   (None, 0.9092784227275393),
   (None, 0.9156067465807742),
   (None, 0.931260239174667),
   (None, 0.93166700993384)])]

In [41]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(
    map(
      lambda x: REDUCE(*x),
      groupbykey(
        flatten(
          map(
            lambda x: MAP(*x),
            RECORDREADER()
          )
        )
      )
    )
  )



# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [42]:
from typing import Iterator

input_numbers = [3, 8, 2, 15, 6, 1, 12, 9, 7]

def RECORDREADER():
  return iter(enumerate(input_numbers))

def MAP(_, row: NamedTuple):
  yield ("max", row)

def REDUCE(key: str, rows: Iterator[NamedTuple]):
  yield (key, max(rows))

# Запуск MapReduce
list(MapReduce(RECORDREADER, MAP, REDUCE))

[('max', 15)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [49]:
def RECORDREADER():
  return iter(enumerate(input_numbers))

def MAP(_, row: NamedTuple):
  yield ("average", row)

def REDUCE(key: str, rows: Iterator[NamedTuple]):
  count = 0
  sum = 0
  for row in rows:
    sum += row
    count += 1
  if count > 0:
    yield key, sum / count
  else:
    yield 

list(MapReduce(RECORDREADER, MAP, REDUCE))

[('average', 7.0)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [59]:
def groupbykey(iterable):
  t = {}
  for k2, v2 in sorted(iterable, key=lambda element: (element[0], element[1])):
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(
    map(
      lambda x: REDUCE(*x),
      groupbykey(
        flatten(
          map(
            lambda x: MAP(*x),
            RECORDREADER()
          )
        )
      )
    )
  )

input_numbers = [('first', 3), ('second', 8), ('first', 2), ('third', 15), ('first', 6), ('third', 1), ('third', 12), ('second', 9), ('second', 7), ('third', 12), ('first', 2)]

def RECORDREADER():
  return iter(input_numbers)

def MAP(key, row: NamedTuple):
  yield key, row

def REDUCE(key: str, rows: Iterator[NamedTuple]):
  yield key, rows

list(MapReduce(RECORDREADER, MAP, REDUCE))

[('first', [2, 2, 3, 6]), ('second', [7, 8, 9]), ('third', [1, 12, 12, 15])]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [69]:
input_data = [
  User(id=1, age=25, gender='female', social_contacts=240),
  User(id=1, age=25, gender='female', social_contacts=240),
  User(id=2, age=25, gender='female', social_contacts=500),
  User(id=3, age=33, gender='female', social_contacts=800),
  User(id=3, age=33, gender='female', social_contacts=800),
  User(id=0, age=55, gender='male', social_contacts=20),
]

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for k2, v2 in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [
    (partition_id, sorted(partition.items(), key=lambda x: x[0]))
    for (partition_id, partition) in enumerate(partitions)
  ]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for userindex, user in enumerate(split):
      yield (userindex, user)

  # Разделение массива между вычислительными узлами
  split_size = int(np.ceil(len(input_data) / maps))
  for i in range(0, len(input_data), split_size):
    yield RECORDREADER(input_data[i : i + split_size])

def MAP_UNIQUE(_, user):
  yield user.id, user

def REDUCE_UNIQUE(_, users):
  yield users[0]

partitioned_output = MapReduceDistributed(
  INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None
)

[(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

6 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=55, social_contacts=20, gender='male'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (1,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=3, age=33, social_contacts=800, gender='female')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [73]:
# Предикат C = проверка параметра gender
def MAP(id_row, row: NamedTuple, predicate):
  if predicate(id_row, row):
    yield row, row

def REDUCE(row, rows: Iterator[NamedTuple]):
  yield row, rows

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

C = lambda _, row_p: row_p.social_contacts > 240
MAP_SELECT = lambda id_row, row: MAP(id_row, row, C)

list(MapReduce(RECORDREADER, MAP_SELECT, REDUCE))

[(User(id=2, age=25, social_contacts=500, gender='female'),
  [User(id=2, age=25, social_contacts=500, gender='female')]),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  [User(id=3, age=33, social_contacts=800, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [78]:
input_collection_copy = [
  dict(id=0, age=55, gender='male', social_contacts=20),
  dict(id=1, age=25, gender='female', social_contacts=240),
  dict(id=2, age=25, gender='female', social_contacts=500),
  dict(id=3, age=33, gender='female', social_contacts=800)
]

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(
    map(
      lambda x: REDUCE(*x),
      groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
    )
  )

# Группировка по id
def MAP(_, row: dict):
  S = [33, 55]
  if row['age'] in S:
    yield (row['id'], row)
  else:
    del row['age']
    yield row['id'], row

def REDUCE(row: str, rows: Iterator[NamedTuple]):
  yield row, rows

def RECORDREADER():
  return [(u['id'], u) for u in input_collection_copy]

list(MapReduce(RECORDREADER, MAP, REDUCE))

[(0, [{'id': 0, 'age': 55, 'gender': 'male', 'social_contacts': 20}]),
 (1, [{'id': 1, 'gender': 'female', 'social_contacts': 240}]),
 (2, [{'id': 2, 'gender': 'female', 'social_contacts': 500}]),
 (3, [{'id': 3, 'age': 33, 'gender': 'female', 'social_contacts': 800}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [82]:
first_collection = [
  User(id=1, age=33, gender="male", social_contacts=11),
  User(id=2, age=25, gender="female", social_contacts=48),
  User(id=3, age=15, gender="female", social_contacts=666),
  User(id=4, age=28, gender="female", social_contacts=1337),
]

second_collection = [
  User(id=5, age=56, gender="male", social_contacts=300),
  User(id=3, age=15, gender="female", social_contacts=666),
  User(id=6, age=19, gender="female", social_contacts=463),
]

# Группировка пользователей по id
def MAP(_, row):
  yield row.id, row

def REDUCE(_, rows):
  yield rows[0], rows[0]

def RECORDREADER():
  return [(u.id, u) for u in first_collection + second_collection]

list(MapReduce(RECORDREADER, MAP, REDUCE))

[(User(id=1, age=33, social_contacts=11, gender='male'),
  User(id=1, age=33, social_contacts=11, gender='male')),
 (User(id=2, age=25, social_contacts=48, gender='female'),
  User(id=2, age=25, social_contacts=48, gender='female')),
 (User(id=3, age=15, social_contacts=666, gender='female'),
  User(id=3, age=15, social_contacts=666, gender='female')),
 (User(id=4, age=28, social_contacts=1337, gender='female'),
  User(id=4, age=28, social_contacts=1337, gender='female')),
 (User(id=5, age=56, social_contacts=300, gender='male'),
  User(id=5, age=56, social_contacts=300, gender='male')),
 (User(id=6, age=19, social_contacts=463, gender='female'),
  User(id=6, age=19, social_contacts=463, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [85]:
def MAP(_, row):
  yield row.id, row

def REDUCE(_, rows):
  if len(rows) == 2:
    yield tuple(rows)

def RECORDREADER():
  return [(u.id, u) for u in first_collection + second_collection]

list(MapReduce(RECORDREADER, MAP, REDUCE))

[(User(id=3, age=15, social_contacts=666, gender='female'),
  User(id=3, age=15, social_contacts=666, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [88]:
def MAP(collection_id, user):
  yield user, collection_id

def REDUCE(user, collections):
  if collections == [False]:
    yield user

def RECORDREADER():
  return [(False, a) for a in first_collection] + [(True, b) for b in second_collection]

list(MapReduce(RECORDREADER, MAP, REDUCE))

[User(id=1, age=33, social_contacts=11, gender='male'),
 User(id=2, age=25, social_contacts=48, gender='female'),
 User(id=4, age=28, social_contacts=1337, gender='female')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [90]:
class User(NamedTuple):
  id: int
  age: int
  social_contacts: int
  gender: str
  car_id: int | None

class Car(NamedTuple):
  id: int
  name: str

first_collection = [
  User(id=1, age=33, gender="male", social_contacts=11, car_id=2),
  User(id=2, age=25, gender="female", social_contacts=48, car_id=3),
  User(id=3, age=15, gender="female", social_contacts=666, car_id=None),
  User(id=4, age=28, gender="female", social_contacts=1337, car_id=1),
]

second_collection = [
  Car(id=1, name='Mercedes'),
  Car(id=2, name='Jigul'),
  Car(id=3, name='Kamaz'),
]

def MAP(car_id, row):
  yield (car_id, row)

def REDUCE(_, rows):
  users = []
  car = None

  for row in rows:
    if type(row) is User:
      users += [row]
    else:
      car = row

  for row in rows:
    if type(row) is User:
      # пользователь, id машины, машина
      yield row, row.car_id, car

def RECORDREADER():
  return [(user.car_id, user) for user in first_collection] + [
    (car.id, car) for car in second_collection
  ]

join_data = list(MapReduce(RECORDREADER, MAP, REDUCE))
join_data

[(User(id=1, age=33, social_contacts=11, gender='male', car_id=2),
  2,
  Car(id=2, name='Jigul')),
 (User(id=2, age=25, social_contacts=48, gender='female', car_id=3),
  3,
  Car(id=3, name='Kamaz')),
 (User(id=3, age=15, social_contacts=666, gender='female', car_id=None),
  None,
  None),
 (User(id=4, age=28, social_contacts=1337, gender='female', car_id=1),
  1,
  Car(id=1, name='Mercedes'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [21]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [22]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs
  
def REDUCE(key, values):
  (i, k) = key
  # solution code that yield(k3,v3) pairs

Проверьте своё решение

In [23]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

TypeError: 'NoneType' object is not iterable

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?